In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import json

import lightgbm as lgb
from sklearn.model_selection import GroupKFold

print("Lightgbm version:", lgb.__version__)

import optuna
from optuna.visualization import (
    plot_edf
    , plot_optimization_history
    , plot_parallel_coordinate
    , plot_param_importances
    , plot_slice
)

print("Optuna version:", optuna.__version__)

# local modules
import sys
sys.path.append("../src")
from preproc import process_train_data

Lightgbm version: 4.5.0


/Users/mavillan/Library/Caches/pypoetry/virtualenvs/mcts-strength-variants-E8z0EJ47-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Optuna version: 4.0.0


***
### load and preprocess data

In [2]:
# define some paths
path_raw = Path("../data/raw")
path_processed = Path("../data/processed")
path_results = Path("../data/results")

# load data
df_train = pd.read_csv(path_raw / "train.csv")
df_test = pd.read_csv(path_raw / "test.csv")

df_train

,Id,GameRulesetName,agent1,agent2,Properties,Format,Time,Discrete,Realtime,Turns,...,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
0,0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",4,0,11,-0.466667
1,1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
2,2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",7,0,8,-0.066667
3,3,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
4,4,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233229,233229,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-NST-false,MCTS-ProgressiveHistory-1.41421356237-Random20...,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",2,0,13,-0.733333
233230,233230,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1-0.6-MAST-false,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",9,1,5,0.266667
233231,233231,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",11,3,1,0.666667
233232,233232,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-true,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",24,2,4,0.666667


In [3]:
# Load the feature selection results
with open('../feat_selection/select_optuna_lgbm.json', 'r') as f:
    feature_selection = json.load(f)

# Extract the selected features
numerical_cols = feature_selection.get('numerical', [])
categorical_cols = feature_selection.get('categorical', [])

print("Numerical features:", len(numerical_cols))
print("Categorical features:", len(categorical_cols))

Numerical features: 281
Categorical features: 6


In [4]:
df_train, numerical_cols, categorical_cols, encoder, scaler = process_train_data(
    df_train,
    scale=False,
    numerical_cols=numerical_cols,
    categorical_cols=categorical_cols
)

# Print the results
print("Numerical Columns:", len(numerical_cols))
print("Categorical Columns:", len(categorical_cols))

number of all nan cols:  0
number of constant cols:  0
Numerical Columns: 281
Categorical Columns: 6


/Users/mavillan/repos/kg/mcts-strength-variants/tuning/../src/preproc.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["utility_agent1_rank"] = (


***
### optimization



In [5]:
# Define the number of folds for cross-validation
num_folds = 5

# Shuffle the train dataset
df_train = df_train.sample(frac=1, random_state=2113).reset_index(drop=True)

# Define the column for stratified or group k-fold
groups_col = 'GameRulesetName'
gkf = GroupKFold(n_splits=num_folds)
split_list = list(gkf.split(df_train, groups=df_train[groups_col]))

In [6]:
def train_and_score(params):
    _params = params.copy()
    num_iterations = _params.pop('num_iterations')

    target = 'utility_agent1'
    oof_scores = []

    # Perform cross-validation
    for _, (train_index, val_index) in enumerate(split_list, 1):
        # Split the data
        X_train, X_val = df_train.iloc[train_index], df_train.iloc[val_index]
        y_train, y_val = X_train[target], X_val[target]
        
        # Create LightGBM datasets
        train_data = lgb.Dataset(
            data=X_train[numerical_cols + categorical_cols],
            label=y_train,
            categorical_feature=categorical_cols,
            free_raw_data=True
        )

        # Train the model
        model = lgb.train(
            _params,
            train_data,
            num_boost_round=num_iterations,
        )

        # predict on validation set
        y_pred = model.predict(X_val[numerical_cols + categorical_cols])
        y_pred = np.clip(y_pred, -1, 1)
        
        # Compute RMSE on scaled values
        rmse = np.sqrt(np.mean((y_pred - y_val) ** 2))
        oof_scores.append(rmse)

    return np.mean(oof_scores)


In [7]:
# Define the parameters
fixed_params = {
    'objective': "regression",
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'bagging_freq': 1,
    'verbose': -1,
    'seed': 2112,
}

def objective(trial):
    # params sampling
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 31, 255),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 100.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 100.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'num_iterations': trial.suggest_int('num_iterations', 1000, 5000),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 100, 1000),
        'min_data_per_group': trial.suggest_int('min_data_per_group', 100, 1000),
        'max_cat_threshold': trial.suggest_int('max_cat_threshold', 32, 512),
        'cat_l2': trial.suggest_float('cat_l2', 1e-8, 100.0, log=True),
        **fixed_params
    }
    
    # Call the train_and_score function
    score = train_and_score(params)
    
    return score

In [8]:
do_optimize = False
timeout = 3600 * 48

study = optuna.create_study(
    study_name="lightgbm.db",
    direction='minimize',
    storage='sqlite:///lightgbm.db',
    load_if_exists=True,
)

if do_optimize:

    study.optimize(
        objective, 
        n_trials=5000, 
        timeout=timeout,
        n_jobs=1, 
        gc_after_trial=True,
    ) 

[I 2024-11-02 00:04:00,248] Using an existing study with name 'lightgbm.db' instead of creating a new one.


In [9]:
study.trials_dataframe().sort_values("value", ascending=True).head(20)

,number,value,datetime_start,datetime_complete,duration,params_bagging_fraction,params_cat_l2,params_feature_fraction,params_lambda_l1,params_lambda_l2,params_max_cat_threshold,params_min_data_in_leaf,params_min_data_per_group,params_num_iterations,params_num_leaves,state
208,208,0.422800,2024-11-01 22:21:39.112370,2024-11-01 22:28:03.040986,0 days 00:06:23.928616,0.998819,0.000060,0.714832,0.477951,0.000365,468,241,963,3170,88,COMPLETE
207,207,0.422918,2024-11-01 22:15:57.468412,2024-11-01 22:21:39.063678,0 days 00:05:41.595266,0.998665,0.000057,0.715260,0.536007,0.000348,469,242,966,3151,77,COMPLETE
200,200,0.423173,2024-11-01 21:36:10.112804,2024-11-01 21:41:18.410699,0 days 00:05:08.297895,0.995375,0.000054,0.710832,0.618089,0.001178,509,266,983,2914,74,COMPLETE
205,205,0.423296,2024-11-01 22:02:38.883957,2024-11-01 22:08:48.546130,0 days 00:06:09.662173,0.999352,0.000057,0.714350,0.410753,0.000658,466,249,971,3167,85,COMPLETE
185,185,0.423348,2024-11-01 20:11:25.327180,2024-11-01 20:16:17.841516,0 days 00:04:52.514336,0.992433,0.000140,0.727442,0.051890,0.000304,469,259,936,2981,69,COMPLETE
196,196,0.423368,2024-11-01 21:13:53.088331,2024-11-01 21:19:39.210688,0 days 00:05:46.122357,0.983440,0.000270,0.712145,0.125257,0.000590,464,265,998,3412,72,COMPLETE
175,175,0.423383,2024-11-01 19:11:01.661975,2024-11-01 19:16:10.552346,0 days 00:05:08.890371,0.987403,0.000089,0.708605,0.061652,0.000287,479,254,984,3112,70,COMPLETE
176,176,0.423642,2024-11-01 19:16:10.598585,2024-11-01 19:21:12.359592,0 days 00:05:01.761007,0.990543,0.000062,0.711640,0.026184,0.000331,483,261,983,3087,69,COMPLETE
186,186,0.423707,2024-11-01 20:16:17.886508,2024-11-01 20:21:22.582678,0 days 00:05:04.696170,0.998880,0.000293,0.726935,0.042912,0.001392,458,261,939,3089,70,COMPLETE
210,210,0.423753,2024-11-01 22:34:10.161799,2024-11-01 22:40:35.579784,0 days 00:06:25.417985,0.999272,0.000071,0.716240,0.357844,0.000379,465,233,981,3177,89,COMPLETE


In [10]:
plot_optimization_history(study)

In [11]:
plot_param_importances(study)

In [12]:
plot_slice(study)


In [13]:
plot_edf(study)


In [14]:
plot_parallel_coordinate(study)


In [15]:
best_params = dict(study.best_params)
best_params = {**fixed_params, **best_params}
best_params

{'objective': 'regression',
 'metric': 'rmse',
 'boosting_type': 'gbdt',
 'learning_rate': 0.05,
 'bagging_freq': 1,
 'verbose': -1,
 'seed': 2112,
 'num_leaves': 88,
 'lambda_l1': 0.4779505822584993,
 'lambda_l2': 0.00036470405010623436,
 'feature_fraction': 0.7148323645686311,
 'bagging_fraction': 0.9988191476198456,
 'num_iterations': 3170,
 'min_data_in_leaf': 241,
 'min_data_per_group': 963,
 'max_cat_threshold': 468,
 'cat_l2': 5.9528678726993896e-05}

***